In [1]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 315 (delta 78), reused 198 (delta 64), pack-reused 0 (from 0)
Receiving objects: 100% (315/315), 9.00 MiB | 14.54 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.0 MB/s eta 0:00:00

In [2]:
import json

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

In [5]:
!pip install auto_gptq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 104.2 MB/s eta 0:00:00


In [25]:
rm_args = {
    "stage": "rm",
    "do_train": True,
    "model_name_or_path": "google/gemma-2b-it",
    "dataset": "ultrafeedback",
    "dataset_dir": "data",
    "template": "gemma",
    "finetuning_type": "lora",
    "flash_attn": "auto",
    "lora_rank": 8,
    "lora_alpha": 16,
    "lora_dropout": 0,
    "lora_target": "all",
    "output_dir": "saves/Gemma-2B-Instruct/lora/gemma_rm_lora",
    "overwrite_output_dir": True,
    "num_train_epochs": 1,
    "max_samples": 1000,
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 8,
    "learning_rate": 1e-5,
    "lr_scheduler_type": "cosine",
    "logging_steps": 10,
    "save_steps": 100,
    "eval_steps": 100,
    "fp16": True,
    "evaluation_strategy": "steps",
    "max_grad_norm": 0.3,
    "val_size": 0.1,
    "preprocessing_num_workers": 16,
    "include_num_input_tokens_seen": True
}

with open("rm_config.json", "w") as f:
    json.dump(rm_args, f, indent=2)



In [26]:
# Run Reward Model training

!CUDA_VISIBLE_DEVICES=0 llamafactory-cli train rm_config.json

2024-10-17 09:01:04.963073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 09:01:04.983062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 09:01:04.989112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 09:01:05.003622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 09:01:06.580572: W tensorflow/comp

# Reward Modeling


In [27]:
!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-10-17 09:01:54.289650: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 09:01:54.322045: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 09:01:54.331955: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 09:01:54.354250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 09:01:55.701035: W tensorflow/comp

In [ ]:
# llamafactory-cli train \
#     --stage rm \
#     --do_train True \
#     --model_name_or_path google/gemma-2b-it \
#     --preprocessing_num_workers 16 \
#     --finetuning_type lora \
#     --template gemma \
#     --flash_attn auto \
#     --dataset_dir data \
#     --dataset ultrafeedback \
#     --cutoff_len 512 \
#     --learning_rate 1e-05 \
#     --num_train_epochs 1.0 \
#     --max_samples 1000 \
#     --per_device_train_batch_size 2 \
#     --gradient_accumulation_steps 8 \
#     --lr_scheduler_type cosine \
#     --max_grad_norm 0.3 \
#     --logging_steps 5 \
#     --save_steps 100 \
#     --warmup_steps 0 \
#     --optim adamw_torch \
#     --packing False \
#     --report_to none \
#     --output_dir saves/Gemma-2B-Instruct/lora/train_2024-10-17-09-02-07 \
#     --fp16 True \
#     --plot_loss True \
#     --ddp_timeout 180000000 \
#     --include_num_input_tokens_seen True \
#     --lora_rank 8 \
#     --lora_alpha 16 \
#     --lora_dropout 0 \
#     --lora_target all \
#     --val_size 0.1 \
#     --eval_strategy steps \
#     --eval_steps 100 \
#     --per_device_eval_batch_size 2

# SFT

In [28]:
!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-10-17 09:18:39.342803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 09:18:39.376270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 09:18:39.386702: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 09:18:39.410486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 09:18:40.837741: W tensorflow/comp

In [ ]:
# llamafactory-cli train \
#     --stage sft \
#     --do_train True \
#     --model_name_or_path google/gemma-2b-it \
#     --preprocessing_num_workers 16 \
#     --finetuning_type lora \
#     --template gemma \
#     --flash_attn auto \
#     --dataset_dir data \
#     --dataset alpaca_en \
#     --cutoff_len 512 \
#     --learning_rate 0.0002 \
#     --num_train_epochs 1.0 \
#     --max_samples 1000 \
#     --per_device_train_batch_size 2 \
#     --gradient_accumulation_steps 8 \
#     --lr_scheduler_type cosine \
#     --max_grad_norm 0.3 \
#     --logging_steps 5 \
#     --save_steps 100 \
#     --warmup_steps 0 \
#     --optim adamw_torch \
#     --packing False \
#     --report_to none \
#     --output_dir saves/Gemma-2B-Instruct/lora/train_2024-10-17-09-18-52 \
#     --fp16 True \
#     --plot_loss True \
#     --ddp_timeout 180000000 \
#     --include_num_input_tokens_seen True \
#     --lora_rank 8 \
#     --lora_alpha 16 \
#     --lora_dropout 0 \
#     --lora_target all \
#     --val_size 0.1 \
#     --eval_strategy steps \
#     --eval_steps 100 \
#     --per_device_eval_batch_size 2

# PPO

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python src/webui.py

2024-10-17 09:52:29.553947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 09:52:29.574470: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 09:52:29.580567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 09:52:29.596359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 09:52:30.889070: W tensorflow/comp

In [ ]:
# llamafactory-cli train \
#     --stage ppo \
#     --do_train True \
#     --model_name_or_path google/gemma-2b-it \
#     --preprocessing_num_workers 16 \
#     --finetuning_type lora \
#     --template gemma \
#     --flash_attn auto \
#     --dataset_dir data \
#     --dataset alpaca_en \
#     --cutoff_len 512 \
#     --learning_rate 1e-05 \
#     --num_train_epochs 1.0 \
#     --max_samples 1000 \
#     --per_device_train_batch_size 2 \
#     --gradient_accumulation_steps 8 \
#     --lr_scheduler_type cosine \
#     --max_grad_norm 0.3 \
#     --logging_steps 5 \
#     --save_steps 100 \
#     --warmup_steps 0 \
#     --optim adamw_torch \
#     --packing False \
#     --report_to none \
#     --output_dir saves/Gemma-2B-Instruct/lora/train_2024-10-17-09-54-12 \
#     --fp16 True \
#     --plot_loss True \
#     --ddp_timeout 180000000 \
#     --include_num_input_tokens_seen True \
#     --adapter_name_or_path saves/Gemma-2B-Instruct/lora/train_2024-10-17-09-18-52 \
#     --lora_rank 8 \
#     --lora_alpha 16 \
#     --lora_dropout 0 \
#     --lora_target all \
#     --reward_model saves/Gemma-2B-Instruct/lora/train_2024-10-17-09-02-07 \
#     --reward_model_type lora \
#     --top_k 0 \
#     --top_p 0.9

In [ ]:
import shutil

shutil.rmtree('/content/LLaMA-Factory/saves/Gemma-2B-Instruct/lora/gemma_rm_lora')